## This is a quick walkthrough of the Laplace approximationa algorithm in inferrence of a posterior distribution of dynamic topic modeling
### [original paper link](http://repository.cmu.edu/cgi/viewcontent.cgi?article=2036&context=compsci)

### Data generation process:
For each topic k at time t=0 <br>
$\beta_{k,0} \sim N(0,I)$ <br>
<br>
For each topic k at time t $>$ 0 <br>
$\beta_{k,t} \sim N(\beta_{k,t-1},\sigma^2 I)$ <br>
<br>
For each document d at time t <br>
$\theta_{t,d} \sim Dirichlet(\alpha)$ <br>
<br>
For each word n at document d at time t<br>
$z_{t,d}^{n} \sim Discrete(\theta_{t,d})$<br>
$w_{t,d}^{n} \sim Discrete(\pi(\beta_{z_{t,d}^{n},t}))$<br>

### We first simulate some fake data per the above data generation process
### For simplicity, we assume each document is of the same length D

In [1]:
import numpy as np
from simulate import *

### some hyper parameters

In [2]:
K = 5
V = 7
N = 10
D = 20
T = 3
sigma = 0.1
alpha_0 = np.array([1]*K)

In [3]:
document_sim = simulate_data(K=K, V=V, N=N, D=D, T=T, sigma = sigma, alpha_0 = alpha_0)
print("simulated data at t=0")
document_sim[0]


topics= 5, vocab = 7, D = 20, N = 10, T =3
simulated data at t=0


array([[ 5.,  4.,  5.,  5.,  4.,  4.,  0.,  4.,  2.,  3.],
       [ 1.,  6.,  2.,  1.,  0.,  1.,  3.,  4.,  4.,  4.],
       [ 3.,  5.,  6.,  4.,  3.,  2.,  1.,  6.,  6.,  6.],
       [ 4.,  4.,  5.,  0.,  1.,  6.,  4.,  0.,  6.,  4.],
       [ 0.,  4.,  5.,  0.,  4.,  3.,  4.,  4.,  4.,  6.],
       [ 4.,  4.,  4.,  4.,  5.,  6.,  5.,  1.,  4.,  5.],
       [ 3.,  4.,  4.,  1.,  1.,  3.,  6.,  6.,  4.,  1.],
       [ 6.,  0.,  3.,  5.,  4.,  5.,  6.,  5.,  0.,  2.],
       [ 4.,  4.,  3.,  3.,  4.,  5.,  3.,  5.,  3.,  0.],
       [ 3.,  6.,  3.,  4.,  4.,  4.,  2.,  5.,  4.,  6.],
       [ 4.,  6.,  6.,  3.,  4.,  6.,  0.,  6.,  2.,  6.],
       [ 4.,  3.,  3.,  6.,  4.,  3.,  5.,  5.,  5.,  1.],
       [ 4.,  4.,  2.,  4.,  4.,  6.,  5.,  3.,  3.,  4.],
       [ 6.,  4.,  5.,  6.,  5.,  0.,  1.,  0.,  0.,  4.],
       [ 3.,  1.,  6.,  1.,  5.,  6.,  4.,  0.,  4.,  4.],
       [ 1.,  6.,  4.,  0.,  4.,  5.,  0.,  4.,  1.,  1.],
       [ 1.,  4.,  2.,  4.,  6.,  2.,  1.,  2.,  6.,  1.

In [4]:
K

5